# Spoiler generation

In [4]:
import pandas as pd
from datasets import load_dataset

from spoiler_generation.utils.stats import prepare_stats, calculate_bleu
from spoiler_generation.utils.dataset_class import Dataset


In [2]:
# The dataset is private
dataset = load_dataset("MateuszW/clickbait_spoiling_test")
test_df = pd.DataFrame(dataset["test"])
test_df["spoiler"] = test_df["spoiler"].apply(Dataset.preprocess_func)

Found cached dataset csv (C:/Users/Mateusz/.cache/huggingface/datasets/MateuszW___csv/MateuszW--clickbait_spoiling_test-aaa35fe962f96287/0.0.0/6954658bab30a358235fa864b05cf819af0e179325c740e4bc853bcc7ec513e1)


  0%|          | 0/1 [00:00<?, ?it/s]

In [31]:
test_df.shape

(1000, 6)

In [3]:
models_outputs = load_dataset(
    "MateuszW/spoiler_generation",
    data_files={
        "baseline": "models_output/deberta-baseline_output.csv",
        "deberta_paqott": "models_output/deberta-paqott_output.csv",
        "llama_pwot": "models_output/llama-pwot_output.csv",
        "vicuna_pwot": "models_output/vicuna-pwot_output.csv",
        "opt_pwot": "models_output/opt-pwot_output.csv",
        "llama_pwt": "models_output/llama-pwt_output.csv",
        "vicuna_pwt": "models_output/vicuna-pwt_output.csv",
        "opt_pwt": "models_output/opt-pwt_output.csv",
        "vicuna_ppt": "models_output/vicuna-ppt_output.csv",
    },
)

Found cached dataset csv (C:/Users/Mateusz/.cache/huggingface/datasets/MateuszW___csv/MateuszW--spoiler_generation-2d60d0350a7a6926/0.0.0/6954658bab30a358235fa864b05cf819af0e179325c740e4bc853bcc7ec513e1)


  0%|          | 0/9 [00:00<?, ?it/s]

In [5]:
for output_name in models_outputs.keys():
    models_outputs[output_name] = pd.DataFrame(models_outputs[output_name])
    models_outputs[output_name]["spoiler"] = models_outputs[output_name]["spoiler"].apply(Dataset.preprocess_func)

In [7]:
for output_name in models_outputs.keys():
    print(output_name, calculate_bleu(test_df, models_outputs[output_name]))

baseline 0.3789261019434709
deberta_paqott 0.38212501050366765
llama_pwot 0.32208982284824633
vicuna_pwot 0.3315958503556814
opt_pwot 0.34201745459592736
llama_pwt 0.37699728737519966
vicuna_pwt 0.3880001338892428
opt_pwt 0.33857424975291617
vicuna_ppt 0.4001146909888649


### Per type

In [ ]:
for spoiler_type in ["phrase", "passage", "multi"]:
    spoiler_type_df = test_df.loc[test_df["type"] == spoiler_type]
    print(spoiler_type, prepare_stats(spoiler_type_df, pd.DataFrame(models_outputs["baseline"])))